# Class 8. Namespace & Scope Rule (LEGB)

## 📌 학습 목표
1. LEGB 규칙을 완벽히 이해하고 설명할 수 있다
2. global과 nonlocal의 차이를 구분할 수 있다
3. Closure의 동작 원리를 이해한다
4. Variable Shadowing 문제를 해결할 수 있다

---

## 🕐 1. Namespace 완전 정복

### 1.1 Namespace = 이름 → 객체 매핑

#### 💡 핵심 개념

**Namespace**: 이름(Name)과 객체(Object)를 연결하는 사전(Dictionary)

```python
# Namespace는 개념적으로 딕셔너리
namespace = {
    'x': <int object at 0x...>,
    'name': <str object at 0x...>,
    'func': <function object at 0x...>
}
```

**비유**:
- **전화번호부**: 이름 → 전화번호
- **Namespace**: 이름 → 객체

In [ ]:
# Namespace의 실체 확인
print("=== Namespace는 Dictionary ===")

x = 10
name = "Alice"

# globals()는 전역 Namespace를 딕셔너리로 반환
print(f"globals()의 타입: {type(globals())}")
print(f"\n'x' in globals(): {'x' in globals()}")
print(f"globals()['x']: {globals()['x']}")
print(f"\n'name' in globals(): {'name' in globals()}")
print(f"globals()['name']: {globals()['name']}")

### 1.2 Namespace의 종류

#### Python의 4가지 Namespace

```
┌─────────────────────────────────────────┐
│  1. Built-in Namespace                  │  ← len, print, int
│     ↓                                   │
│  2. Global Namespace                    │  ← 모듈 레벨 변수
│     ↓                                   │
│  3. Enclosing Namespace                 │  ← 중첩 함수의 외부
│     ↓                                   │
│  4. Local Namespace                     │  ← 함수 내부 변수
└─────────────────────────────────────────┘
```

**1. Built-in Namespace**
- Python 시작 시 생성
- `len`, `print`, `int` 등 내장 함수/타입
- 모든 곳에서 접근 가능

**2. Global Namespace**
- 모듈(파일) 레벨에서 생성
- 모듈이 import될 때 생성
- 모듈 전체에서 접근 가능

**3. Enclosing Namespace**
- 중첩 함수의 외부 함수에서 생성
- Closure에서 중요

**4. Local Namespace**
- 함수 호출 시 생성
- 함수 종료 시 소멸
- 함수 내부에서만 접근

In [ ]:
# 4가지 Namespace 확인
print("=== 4가지 Namespace ===")

# 1. Built-in Namespace
import builtins
print("1. Built-in Namespace:")
builtin_names = [name for name in dir(builtins) if not name.startswith('_')][:5]
print(f"   예: {builtin_names}...")

# 2. Global Namespace
global_var = "I'm global"
print(f"\n2. Global Namespace:")
print(f"   global_var in globals(): {'global_var' in globals()}")

# 3 & 4. Enclosing & Local
def outer():
    enclosing_var = "I'm enclosing"
    
    def inner():
        local_var = "I'm local"
        print(f"\n3. Enclosing Namespace:")
        print(f"   enclosing_var: {enclosing_var}")
        print(f"\n4. Local Namespace:")
        print(f"   locals(): {locals()}")
    
    inner()

outer()

### 1.3 Namespace Lifecycle

#### 생성과 소멸 시점

| Namespace | 생성 시점 | 소멸 시점 |
|-----------|-----------|----------|
| Built-in | Python 인터프리터 시작 | 인터프리터 종료 |
| Global | 모듈 import 또는 실행 | 프로그램 종료 |
| Enclosing | 외부 함수 호출 | 외부 함수 종료 |
| Local | 함수 호출 | 함수 종료 |

In [1]:
# Namespace Lifecycle
import sys

print("=== Namespace Lifecycle ===")

def demo_lifecycle():
    local_x = 100
    print(f"함수 내부:")
    print(f"  'local_x' in locals(): {'local_x' in locals()}")
    print(f"  local_x의 refcount: {sys.getrefcount(local_x) - 1}")

print("함수 호출 전:")
# print(f"  local_x 존재? {local_x}")  # NameError!

demo_lifecycle()

print("\n함수 호출 후:")
print("  local_x는 소멸됨 (NameError 발생)")
# print(f"  local_x 존재? {local_x}")  # NameError!

=== Namespace Lifecycle ===
함수 호출 전:
함수 내부:
  'local_x' in locals(): True
  local_x의 refcount: 4294967294

함수 호출 후:
  local_x는 소멸됨 (NameError 발생)


### 1.4 접근 함수: globals(), locals(), vars()

#### Namespace 접근 도구

In [2]:
# globals(), locals(), vars()
print("=== Namespace 접근 함수 ===")

global_x = 10

def demo():
    local_y = 20
    
    # 1. globals(): 항상 전역 Namespace
    print("1. globals():")
    print(f"   'global_x' in globals(): {'global_x' in globals()}")
    print(f"   'local_y' in globals(): {'local_y' in globals()}")
    
    # 2. locals(): 현재 Local Namespace
    print("\n2. locals():")
    print(f"   'local_y' in locals(): {'local_y' in locals()}")
    print(f"   'global_x' in locals(): {'global_x' in locals()}")
    print(f"   locals(): {locals()}")
    
    # 3. vars(): 객체의 __dict__
    print("\n3. vars() == locals():")
    print(f"   {vars() == locals()}")

demo()

# 전역에서 locals() == globals()
print("\n전역 레벨:")
print(f"locals() is globals(): {locals() is globals()}")

=== Namespace 접근 함수 ===
1. globals():
   'global_x' in globals(): True
   'local_y' in globals(): False

2. locals():
   'local_y' in locals(): True
   'global_x' in locals(): False
   locals(): {'local_y': 20}

3. vars() == locals():
   True

전역 레벨:
locals() is globals(): True


#### vars() 활용

**함수 시그니처**: `vars([object])`

- 인자 없음: `locals()` 반환
- 인자 있음: 객체의 `__dict__` 반환

In [3]:
# vars() 활용
print("=== vars() 활용 ===")

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

person = Person("Alice", 25)

# 객체의 속성 확인
print(f"vars(person): {vars(person)}")
print(f"person.__dict__: {person.__dict__}")
print(f"\nvars(person) == person.__dict__: {vars(person) == person.__dict__}")

=== vars() 활용 ===
vars(person): {'name': 'Alice', 'age': 25}
person.__dict__: {'name': 'Alice', 'age': 25}

vars(person) == person.__dict__: True


---

## 🕑 2. LEGB Rule

### 2.1 LEGB = Name Resolution 순서

#### 💡 핵심: 이름을 찾는 순서

```
L (Local)      → E (Enclosing) → G (Global) → B (Built-in)
   ↓                  ↓               ↓            ↓
함수 내부        외부 함수        모듈 레벨     Python 내장
```

**규칙**:
1. **Local**부터 시작
2. 없으면 **Enclosing** 검색
3. 없으면 **Global** 검색
4. 없으면 **Built-in** 검색
5. 여전히 없으면 **NameError**

In [ ]:
# LEGB Rule 기본
print("=== LEGB Rule ===")

x = 'Global x'

def outer():
    x = 'Enclosing x'
    
    def inner():
        x = 'Local x'
        print(f"inner() - x: {x}")
        print("  → Local에서 찾음 (L)")
    
    inner()
    print(f"\nouter() - x: {x}")
    print("  → Local에서 찾음 (Enclosing Scope의 Local)")

outer()
print(f"\nglobal - x: {x}")
print("  → Global에서 찾음 (G)")

print(f"\nBuilt-in - len: {len}")
print("  → Built-in에서 찾음 (B)")

### 2.2 각 Scope 상세

#### L (Local Scope)

In [ ]:
# Local Scope
print("=== L (Local Scope) ===")

def func():
    # 함수 내부에서 정의된 모든 변수
    local_var = 100
    
    # Parameter도 Local Scope
    def with_param(param):
        print(f"param: {param} (Local)")
        print(f"'param' in locals(): {'param' in locals()}")
    
    with_param(42)
    
    # for 문의 변수도 Local Scope
    for i in range(3):
        pass
    print(f"\nfor 문 후 i: {i} (여전히 Local)")

func()

#### E (Enclosing Scope)

In [ ]:
# Enclosing Scope
print("=== E (Enclosing Scope) ===")

def outer():
    enclosing_var = "I'm in enclosing"
    
    def middle():
        middle_var = "I'm in middle"
        
        def inner():
            # Enclosing: middle() 또는 outer()
            print(f"inner에서:")
            print(f"  enclosing_var: {enclosing_var}")
            print(f"  middle_var: {middle_var}")
            print("  → 둘 다 Enclosing Scope")
        
        inner()
    
    middle()

outer()

#### G (Global Scope)

In [ ]:
# Global Scope
print("=== G (Global Scope) ===")

# 모듈 레벨 변수
module_var = "I'm global"
PI = 3.14159

def access_global():
    # Global 변수 읽기는 자유
    print(f"module_var: {module_var}")
    print(f"PI: {PI}")
    
    # 하지만 수정은...
    # module_var = "Modified"  # 이건 Local 생성!
    # PI += 1  # UnboundLocalError!

access_global()
print(f"\n함수 외부:")
print(f"module_var: {module_var} (변경 안 됨)")

#### B (Built-in Scope)

In [ ]:
# Built-in Scope
print("=== B (Built-in Scope) ===")

import builtins

# Python 내장
print(f"len: {len}")
print(f"int: {int}")
print(f"print: {print}")

# Built-in Namespace 확인
print(f"\n'len' in dir(builtins): {'len' in dir(builtins)}")

# 주의: Built-in도 Shadowing 가능!
def shadow_builtin():
    len = "I'm not the len you know"
    print(f"\nshadow_builtin() - len: {len}")
    # print(len([1, 2, 3]))  # TypeError!

shadow_builtin()
print(f"외부 - len: {len} (여전히 Built-in)")

### 2.3 LEGB 종합 예제

In [ ]:
# LEGB 종합 예제
print("=== LEGB 종합 ===")

# Built-in
# len은 Built-in

# Global
x = 'Global'
y = 'Global only'

def outer():
    # Enclosing
    x = 'Enclosing'
    z = 'Enclosing only'
    
    def inner():
        # Local
        x = 'Local'
        w = 'Local only'
        
        print("inner() 내부:")
        print(f"  x: {x}           (L)")
        print(f"  z: {z}  (E)")
        print(f"  y: {y}      (G)")
        print(f"  len: {len}         (B)")
        print(f"  w: {w}       (L)")
    
    inner()
    print(f"\nouter() 내부:")
    print(f"  x: {x}  (L of outer = E of inner)")

outer()
print(f"\nGlobal:")
print(f"  x: {x}")

---

## 🕒 3. global & nonlocal

### 3.1 global 키워드

#### 목적: Local에서 Global 수정

**문제**:
```python
count = 0

def increment():
    count = count + 1  # UnboundLocalError!
    # Python은 할당을 보고 count를 Local로 판단
```

**해결**:
```python
count = 0

def increment():
    global count  # Global count를 사용
    count = count + 1
```

In [ ]:
# global 키워드
print("=== global 키워드 ===")

counter = 0

def increment():
    global counter
    counter += 1
    print(f"함수 내부 - counter: {counter}")

print(f"초기값: {counter}")
increment()
increment()
increment()
print(f"최종값: {counter}")

# global 없이 시도
def wrong_increment():
    # counter += 1  # UnboundLocalError!
    pass

print("\nglobal 없으면 UnboundLocalError 발생")

#### global의 위험성

**문제점**:
1. **예측 불가능**: 여러 함수가 전역 변수 수정
2. **테스트 어려움**: 전역 상태에 의존
3. **버그 추적 어려움**: 어디서 변경되었는지 불명확

**대안**:
- 함수의 반환값 사용
- 클래스 사용
- Closure 사용

In [ ]:
# global 대안
print("=== global 대안 ===")

# ❌ global 사용
total = 0

def add_bad(value):
    global total
    total += value

# ✅ 반환값 사용
def add_good(current, value):
    return current + value

# 비교
add_bad(10)
print(f"global 사용: {total}")

result = 0
result = add_good(result, 10)
print(f"반환값 사용: {result}")

### 3.2 nonlocal 키워드

#### 목적: Local에서 Enclosing 수정

**용도**: Closure에서 외부 함수 변수 수정

```python
def outer():
    count = 0
    
    def inner():
        nonlocal count  # Enclosing count 사용
        count += 1
        return count
    
    return inner
```

In [ ]:
# nonlocal 키워드
print("=== nonlocal 키워드 ===")

def make_counter():
    count = 0
    
    def increment():
        nonlocal count
        count += 1
        return count
    
    return increment

counter1 = make_counter()
counter2 = make_counter()

print("counter1:", counter1(), counter1(), counter1())
print("counter2:", counter2(), counter2())
print("\n각 카운터는 독립적!")

#### global vs nonlocal

| 특징 | global | nonlocal |
|------|--------|----------|
| 대상 | Global Scope | Enclosing Scope |
| 용도 | 전역 변수 수정 | 외부 함수 변수 수정 |
| 위치 | 어디서나 | 중첩 함수에서만 |
| 권장 | ❌ 피하기 | ✅ Closure에서 유용 |

In [ ]:
# global vs nonlocal
print("=== global vs nonlocal ===")

x = 'Global x'

def outer():
    x = 'Enclosing x'
    
    def use_global():
        global x
        print(f"use_global: {x} (Global)")
    
    def use_nonlocal():
        nonlocal x
        print(f"use_nonlocal: {x} (Enclosing)")
    
    use_global()
    use_nonlocal()

outer()

### 3.3 Variable Shadowing

#### 정의: 같은 이름이 여러 Scope에 존재

**Shadowing**: 내부 Scope의 변수가 외부 변수를 "가림"

In [ ]:
# Variable Shadowing
print("=== Variable Shadowing ===")

x = 'Outer'

def func():
    x = 'Inner'  # Outer x를 Shadow
    print(f"func() - x: {x}")

print(f"Before: {x}")
func()
print(f"After: {x}")
print("\n→ Outer x는 영향 받지 않음")

#### Shadowing의 위험성

In [ ]:
# Shadowing 위험 예제
print("=== Shadowing 위험 ===")

# Built-in Shadowing (매우 위험!)
def dangerous():
    list = [1, 2, 3]  # Built-in list를 Shadow
    print(f"list 변수: {list}")
    
    # list() 함수 사용 불가!
    # numbers = list(range(5))  # TypeError!
    print("list() 함수를 사용할 수 없음!")

dangerous()

# 함수 밖에서는 정상
numbers = list(range(5))
print(f"\n함수 외부: list(range(5)) = {numbers}")

#### Shadowing 해결 방법

1. **명확한 이름 사용**
2. **Built-in 이름 피하기**
3. **global/nonlocal 활용**

In [ ]:
# Shadowing 해결
print("=== Shadowing 해결 ===")

# ❌ 나쁜 예
def bad_naming():
    list = [1, 2, 3]  # Built-in shadow
    str = "hello"     # Built-in shadow
    print("Built-in을 사용할 수 없음")

# ✅ 좋은 예
def good_naming():
    my_list = [1, 2, 3]     # 명확한 이름
    greeting = "hello"       # 명확한 이름
    numbers = list(range(5)) # Built-in 사용 가능
    print(f"numbers: {numbers}")

good_naming()

### 3.4 Closure 기초

#### Closure = Enclosing Namespace 포획

**정의**: 함수가 자신이 생성될 때의 Enclosing Scope를 "기억"

**조건**:
1. 중첩 함수
2. 외부 함수의 변수 사용
3. 외부 함수가 내부 함수 반환

In [ ]:
# Closure 기본
print("=== Closure 기본 ===")

def make_multiplier(n):
    # n은 Enclosing 변수
    
    def multiplier(x):
        # n을 "기억"
        return x * n
    
    return multiplier

# 서로 다른 Closure 생성
double = make_multiplier(2)
triple = make_multiplier(3)

print(f"double(5): {double(5)}")
print(f"triple(5): {triple(5)}")

# Closure 확인
print(f"\ndouble의 Closure: {double.__closure__}")
print(f"기억하는 값: {double.__closure__[0].cell_contents}")

#### Closure 활용 패턴

In [ ]:
# Closure 활용: 상태 유지
print("=== Closure로 상태 유지 ===")

def make_accumulator():
    total = 0
    
    def add(value):
        nonlocal total
        total += value
        return total
    
    return add

acc1 = make_accumulator()
acc2 = make_accumulator()

print("acc1:", acc1(10), acc1(20), acc1(30))
print("acc2:", acc2(5), acc2(15))
print("\n각 accumulator는 독립적인 상태 유지")

In [ ]:
# Closure 활용: 함수 공장
print("=== Closure로 함수 공장 ===")

def make_power(exponent):
    def power(base):
        return base ** exponent
    return power

square = make_power(2)
cube = make_power(3)

print(f"square(5): {square(5)}")
print(f"cube(5): {cube(5)}")

---

## 📊 종합 실습

### 실습 1: LEGB 완전 정복

In [ ]:
# 종합 실습 1: LEGB
print("=== LEGB 종합 실습 ===")

x = 'G'
y = 'G only'

def level1():
    x = 'E1'
    z = 'E1 only'
    
    def level2():
        x = 'E2'
        w = 'E2 only'
        
        def level3():
            x = 'L'
            
            print("level3():")
            print(f"  x: {x} (L)")
            print(f"  w: {w} (E - level2)")
            print(f"  z: {z} (E - level1)")
            print(f"  y: {y} (G)")
            print(f"  len: {len} (B)")
        
        level3()
    
    level2()

level1()

### 실습 2: Closure 활용

In [ ]:
# 종합 실습 2: Closure로 은행 계좌
print("=== Closure 실습: 은행 계좌 ===")

def create_account(initial_balance):
    balance = initial_balance
    
    def deposit(amount):
        nonlocal balance
        balance += amount
        return balance
    
    def withdraw(amount):
        nonlocal balance
        if balance >= amount:
            balance -= amount
            return balance
        else:
            return "잔액 부족"
    
    def get_balance():
        return balance
    
    return {
        'deposit': deposit,
        'withdraw': withdraw,
        'balance': get_balance
    }

# 테스트
account = create_account(1000)
print(f"초기 잔액: {account['balance']()}")
print(f"입금 후: {account['deposit'](500)}")
print(f"출금 후: {account['withdraw'](300)}")
print(f"최종 잔액: {account['balance']()}")

---

## 🎯 체크리스트

### Namespace
- [ ] Namespace = 이름→객체 매핑 이해
- [ ] 4가지 Namespace 구분 가능
- [ ] globals(), locals(), vars() 사용
- [ ] Namespace Lifecycle 이해

### LEGB Rule
- [ ] LEGB 순서 외움 (L→E→G→B)
- [ ] 각 Scope 설명 가능
- [ ] Name Resolution 과정 이해
- [ ] Shadowing 개념 이해

### global & nonlocal
- [ ] global 키워드 사용법
- [ ] nonlocal 키워드 사용법
- [ ] global vs nonlocal 차이
- [ ] global 사용 피하는 방법

### Closure
- [ ] Closure 정의 설명 가능
- [ ] Closure 만들 수 있음
- [ ] `__closure__` 확인 가능
- [ ] Closure 활용 패턴 이해

---

## 💡 핵심 요약

### LEGB Rule
```python
# L (Local) → E (Enclosing) → G (Global) → B (Built-in)
x = 'Global'

def outer():
    x = 'Enclosing'
    
    def inner():
        x = 'Local'
        print(x)  # Local 사용
```

### global vs nonlocal
```python
x = 0  # Global

def outer():
    x = 0  # Enclosing
    
    def use_global():
        global x  # Global x 수정
    
    def use_nonlocal():
        nonlocal x  # Enclosing x 수정
```

### Closure
```python
def outer(n):
    def inner(x):
        return x * n  # n을 기억
    return inner

double = outer(2)  # Closure 생성
```

---

## 📚 추가 학습 자료

- Scope LEGB: https://ds31x.tistory.com/45
- Namespace, Frame, Context: https://dsaint31.tistory.com/508
- Function Call Stack: https://dsaint31.me/mkdocs_site/CE/ch05/ch05_02_01_function/#function-call-stack
- Python Scopes: https://docs.python.org/ko/3/tutorial/classes.html#python-scopes-and-namespaces

---

## 🤔 토론 주제

### 1. "global 사용이 나쁜 이유는?"

**문제점:**
- 예측 불가능한 동작
- 테스트 어려움
- 버그 추적 어려움

### 2. "Closure vs Class"

**비교:**
- Closure: 간단한 상태 유지
- Class: 복잡한 상태와 행위

### 3. "Built-in Shadowing은 왜 허용되나?"

**이유:**
- Python의 유연성
- 하지만 위험함
- 피해야 할 패턴

---

## 수고하셨습니다! 🎉